In [2]:
# расскоментируйте код ниже, чтобы установить все зависимости
!pip install -q \
    pyarrow==12.0.1 \
    polars==0.18.6 \
    tqdm==4.65.0 \
    scipy==1.10.1 \
    scikit-learn==1.3.0 \
    numpy==1.24.3 \
    qdrant-client==1.3.1 \
    faiss-cpu==1.7.4 \
    redis==4.6.0 \
    implicit==0.7.0

In [3]:
# раскоментируйте код ниже, чтобы скачать данные
!wget -q https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q ml-100k.zip

In [4]:
import time
import redis
import faiss
import implicit

import polars as pl
import numpy as np
from tqdm import tqdm

import scipy.sparse as sp
from sklearn.model_selection import train_test_split

import random
from typing import List, Any

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## MovieLens датасет

В качестве данных будем использовать датасет с оценками к фильмам Movielens-100k. В нем есть поле ratings, в качестве позитивных событий мы будем считать то, что пользователь поставил оценку > 3 (такое правило принято в статьях, работающих с этим датасетом)

In [71]:
ratings = pl.read_csv(
    'ml-100k/u.data',
    separator='\t',
    has_header=False,
    new_columns=['user_id', 'item_id', 'rating', 'timestamp']
)
# в качестве позитивной реакции возьмем оценки больше 3
ratings = ratings.filter(pl.col('rating') > 3)
ratings

user_id,item_id,rating,timestamp
i64,i64,i64,i64
298,474,4,884182806
253,465,5,891628467
286,1014,5,879781125
200,222,5,876042340
122,387,5,879270459
291,1042,4,874834944
119,392,4,886176814
167,486,4,892738452
299,144,4,877881320


In [73]:
ratings['rating'].value_counts()

rating,counts
i64,u32
4,34174
5,21201


In [74]:
ratings['user_id'].n_unique(), ratings['item_id'].n_unique()

(942, 1447)

In [75]:
# возьмем последние 5% по времени в качестве отложенной выборки
# все что до этого момента, будем использовать для обучения модели
ts_threshold = ratings['timestamp'].quantile(0.95)
holdout_set = ratings.filter(pl.col('timestamp') >= ts_threshold)
ratings = ratings.filter(pl.col('timestamp') < ts_threshold)

In [76]:
ratings['user_id'].n_unique(), ratings['item_id'].n_unique()

(911, 1415)

не все объекты и пользователи попали в датасет для обучения модели коллаборативной фильтрации. Значит будет проблема холодных пользователей и объектов

In [77]:
RANDOM_STATE = 42
TOP_K = 10


def set_seed():
    random.seed(RANDOM_STATE)
    np.random.seed(RANDOM_STATE)


def fit_als(ratings: pl.DataFrame, als_params={"factors": 64}):
    """Метод обучает модель ALS и возвращает эмбеддинги пользователей и объектов
    Про ALS мы поговорим в следующем уроке, а пока что воспользуемся реализацией из библиотеки implicit

    :param ratings: датафрейм с рейтингами
    :param als_params: параметры для обучения модели ALS
    :return: (user_embeddings, item_embeddings)
    """
    set_seed()

    # соберем разреженную матрицу рейтингов
    rows = ratings["user_id"].to_numpy()
    cols = ratings["item_id"].to_numpy()
    values = ratings["rating"].to_numpy() if "rating" in ratings else np.ones_like(rows)
    user_item_data = sp.csr_matrix((values, (rows, cols)))

    # обучим модель ALS
    als_params.setdefault("random_state", RANDOM_STATE)
    # если есть gpu, используем его для ускорения
    als_params.setdefault("use_gpu", implicit.gpu.HAS_CUDA)
    
    model = implicit.als.AlternatingLeastSquares(**als_params)
    model.fit(user_item_data)

    if als_params['use_gpu']:
        return model.user_factors.to_numpy(), model.item_factors.to_numpy()

    return model.user_factors, model.item_factors


def get_recommendations(user_embs: np.array, item_embs: np.array, k: int = TOP_K):
    # строим индекс объектов
    index = faiss.IndexFlatIP(item_embs.shape[1])
    index.add(item_embs)

    # строим рекомендации с помощью dot-product расстояния
    # с запасом, чтобы после фильтрации просмотренных осталось как минимум TOP_K
    return index.search(user_embs, TOP_K * 3)

## Метрики качества

In [79]:
def user_hitrate(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: релевантные объекты
    :param y_rec: рекомендованные объекты
    :param k: число рекомендации для показа (top-K результаты)
    :return: 1, если top-k рекомендации содержат как минимум один релевантный объект
    """
    return int(len(set(y_rec[:k]).intersection(set(y_rel))) > 0)


# метод для оценки качества на отложенной выборке
# можно сначала посмотреть следующие ячейки, чтобы понять, что тут происходит
def evaluate_holdout_set(
    train_df: pl.DataFrame, user_embs: np.array, item_embs: np.array, k: int = TOP_K
):
    # строим индекс эмбеддингов объектов
    index = faiss.IndexFlatIP(item_embs.shape[1])
    index.add(item_embs)

    hitrate_list = []
    hitrate_list_by_type = {"warm": [], "cold": []}

    # датафрейм с колонками user_id, train_item_ids, test_item_ids
    # нам интересно провалидировать только по тем пользователям, которые есть в отложенной выборке
    # поэтому train_item_ids может быть пустым
    grouped_user_items = (
        holdout_set.groupby("user_id")
        .agg(pl.col("item_id").alias("test_item_ids"))
        .join(
            train_df.groupby("user_id").agg(pl.col("item_id").alias("train_item_ids")),
            "user_id",
            "left",
        )
    )

    # предподсчитаем рекомендации по эмбеддингам пользователей и объектов (только для warm пользователей)
    _, recs = get_recommendations(user_embs, item_embs)

    for user_id, test_ids, train_ids in grouped_user_items.rows():
        if train_ids:
            # если есть история для пользователя, то используем предподсчитанные рекомендации
            user_history = train_ids

            y_rel = test_ids
            y_rec = [
                item_id for item_id in recs[user_id] if item_id not in user_history
            ]
            hitrate_list_by_type["warm"].append(user_hitrate(y_rel, y_rec, k))
        else:
            # если нет истории пользователя, то используем сумму эмбеддингов объектов
            user_emb = np.zeros(item_embs.shape[1])
            user_history = set()
            h_r = []
            for i, item_ind in enumerate(test_ids[:-1]):
                user_emb += item_embs[item_ind]
                user_history.add(item_ind)
                
                # хотим порекомендовать следующий объект
                y_rel = [test_ids[i + 1]]
                y_rec = [
                    item_id
                    for item_id in index.search(user_emb[np.newaxis, :], k + 1)[1][0]
                    if item_id not in user_history
                ]
                hitrate_list_by_type["cold"].append(user_hitrate(y_rel, y_rec, k))
            
    all_users_hitrate = np.mean(hitrate_list_by_type["warm"] + hitrate_list_by_type["cold"])
    warm_users_hitrate = np.mean(hitrate_list_by_type["warm"]) if hitrate_list_by_type["warm"] else 1.0
    cold_users_hitrate = np.mean(hitrate_list_by_type["cold"]) if hitrate_list_by_type["cold"] else 1.0
    return {
        "all": all_users_hitrate,
        "warm": warm_users_hitrate,
        "cold": cold_users_hitrate,
    }

In [80]:
assert user_hitrate([1, 2, 3], [4, 5, 2], 3) == 1
assert user_hitrate([1, 2, 3], [4, 5, 2], 2) == 0
assert user_hitrate([1, 2, 3], [4, 5, 6], 10) == 0

## Случайная вадидация

Можно сказать, что в валидацию попадает какой-то процент взаимодействий, а в тренировочную выборку все остальное. С точки зрения реализации самый простой способ, однако совсем никак не контролирует дата-лики, когда информация из будущего оказывается в прошлом

In [81]:
TEST_SIZE = 0.1

train_df, test_df = train_test_split(
    ratings,
    # будем сэмплировать так, чтобы пользователи встречались с той же вероятностью
    stratify=ratings['user_id'],
    test_size=TEST_SIZE,
    # зафиксируем генератор случайных чисел для воспроизводимости результатов
    random_state=RANDOM_STATE,
)

In [82]:
user_embs, item_embs = fit_als(train_df)
probs, recs = get_recommendations(user_embs, item_embs)

100%|██████████| 15/15 [00:06<00:00,  2.29it/s]


In [83]:
# сгруппируем в список индексов объекты из тренировочной выборки и валидационной
train_user_items = train_df.groupby("user_id").agg(
    pl.col("item_id").alias("train_item_ids")
)
test_user_items = test_df.groupby("user_id").agg(
    pl.col("item_id").alias("test_item_ids")
)
# объединим все в одну табличку, при этом важно оставить всех тех
# пользователей, которые есть в валидационной выборке
grouped_user_items = test_user_items.join(train_user_items, "user_id", "left")
grouped_user_items

user_id,test_item_ids,train_item_ids
i64,list[i64],list[i64]
98,[25],"[435, 517, … 523]"
230,"[204, 117, … 125]","[132, 69, … 549]"
290,"[22, 143, … 222]","[183, 164, … 472]"
48,"[170, 132, … 50]","[483, 479, … 511]"
150,"[288, 246, 410]","[235, 268, … 291]"
858,[689],"[9, 127, … 754]"
310,[845],"[304, 24, … 50]"
620,"[379, 281, … 404]","[1503, 699, … 623]"
854,"[238, 979, … 606]","[507, 250, … 9]"


In [84]:
hitrate_list = []
for user_id, user_history, y_rel in grouped_user_items.rows():
    # строим рекомендации из тех объектов, которые уже не были в тренировочной выборке
    y_rec = [item_id for item_id in recs[user_id] if item_id not in user_history]
    hitrate_list.append(user_hitrate(y_rel, y_rec, TOP_K))
print(f'Hitrate@{TOP_K} = {np.mean(hitrate_list)}')

Hitrate@10 = 1.0


In [85]:
evaluate_holdout_set(train_df, user_embs, item_embs)

{'all': 0.06108597285067873,
 'warm': 0.21739130434782608,
 'cold': 0.05473808122424956}

На валидации получили идеальную метрику hitrate, однако на отложенной выборке самый худший результат по сравнению со следующими методами валидации

## Валидация по пользователям

Здесь мы делаем разделение данных уже по полю user_id, то есть какие-то пользователи попадут только в тестовую выборку и с точки зрения алгоритма будут холодными (cold) пользователями

Для того, чтобы построить рекомендации для таких пользователей, будем использовать эмбеддинги объектов в реальном времени. То есть если для валидации у нас есть список объектов [item_1, item_2, item_3, ...], то для первого объекта ничего не рекомендуем (или еще лучше рекомендовать популярные объекты). В момент рекомендации второго объекта у нас есть история для пользователя: [item_1], тогда эмбединг пользователя равен эмбеддингу item_1. Для следующего объекта эмбеддинг будет равен сумме эмбеддингов item_1 и item_2 и так далее..

In [86]:
# зафиксируем генератор случайных чисел для воспроизводимости
np.random.seed(RANDOM_STATE)  

# выберем среди всех пользователей тех, кто будет в тренировочной выборке, а кто в тестовой
unique_users = ratings["user_id"].unique().to_list()
test_users = set(
    np.random.choice(unique_users, int(len(unique_users) * TEST_SIZE), replace=False)
)
train_users = set(unique_users).difference(test_users)

train_df = ratings.filter(pl.col("user_id").is_in(train_users))
test_df = ratings.filter(pl.col("user_id").is_in(test_users))

# sanity check
assert set(train_df["user_id"].unique().to_list()) == train_users
assert set(test_df["user_id"].unique().to_list()) == test_users
assert len(train_df) + len(test_df) == len(ratings)

In [87]:
user_embs, item_embs = fit_als(train_df)
probs, recs = get_recommendations(user_embs, item_embs)

100%|██████████| 15/15 [00:06<00:00,  2.28it/s]


In [91]:
# сгруппируем в список индексов объекты из тренировочной выборки и валидационной
train_user_items = train_df.groupby("user_id").agg(
    pl.col("item_id").alias("train_item_ids")
)
test_user_items = test_df.groupby("user_id").agg(
    pl.col("item_id").alias("test_item_ids")
)
# объединим все в одну табличку, при этом важно оставить всех тех
# пользователей, которые есть в валидационной выборке
grouped_user_items = test_user_items.join(train_user_items, "user_id", "left")
grouped_user_items

user_id,test_item_ids,train_item_ids
i64,list[i64],list[i64]
572,"[300, 476, … 813]",null
76,"[61, 175, … 96]",null
66,"[298, 258, … 300]",null
78,"[255, 412, … 411]",null
748,"[748, 22, … 199]",null
26,"[125, 15, … 288]",null
148,"[408, 1, … 194]",null
34,"[312, 242, … 991]",null
318,"[474, 356, … 340]",null


In [92]:
# строим индекс объектов
index = faiss.IndexFlatIP(item_embs.shape[1])
index.add(item_embs)
    
hitrate_list = []
for _, user_session, _ in grouped_user_items.rows():
    user_emb = np.zeros(item_embs.shape[1])
    user_history = set()
    # эмбеддинг пользователя - сумма эмбеддингов позитивных объектов
    # пройдемся по каждому объекту и постараемся предсказать следующие в сессии
    for i, item_ind in enumerate(user_session[:-1]):
        user_emb += item_embs[item_ind]
        user_history.add(item_ind)
        
        y_rel = [user_session[i + 1]]
        y_rec = [
            item_id 
            for item_id in index.search(user_emb[np.newaxis, :], TOP_K * 3)[1][0]
            if item_id not in user_history
        ]
    
        hitrate_list.append(user_hitrate(y_rel, y_rec, TOP_K))
print(f'Hitrate@{TOP_K} = {np.mean(hitrate_list)}')

Hitrate@10 = 0.11200508151598561


In [93]:
evaluate_holdout_set(train_df, user_embs, item_embs)

{'all': 0.06195175438596491,
 'warm': 0.27419354838709675,
 'cold': 0.05448354143019296}

## Валидация по времени

Такая валидация дублирует реально поведение системы, когда у нас есть данные только из прошлого и мы хотим порекомендовать для будущего

In [94]:
ts_threshold = ratings['timestamp'].quantile(1 - TEST_SIZE)
train_df = ratings.filter(pl.col('timestamp') < ts_threshold)
test_df = ratings.filter(pl.col('timestamp') >= ts_threshold)

assert len(train_df) + len(test_df) == len(ratings)

In [95]:
user_embs, item_embs = fit_als(train_df)
probs, recs = get_recommendations(user_embs, item_embs)

100%|██████████| 15/15 [00:05<00:00,  2.61it/s]


In [96]:
train_user_items = train_df.groupby('user_id').agg(
    pl.col('item_id').alias('train_item_ids')
)
test_user_items = test_df.groupby('user_id').agg(
    pl.col('item_id').alias('test_item_ids')
)
grouped_user_items = test_user_items.join(train_user_items, 'user_id', 'left')
grouped_user_items

user_id,test_item_ids,train_item_ids
i64,list[i64],list[i64]
642,"[845, 993, 245]","[1039, 38, … 73]"
358,"[469, 896, … 1005]",null
938,"[864, 313, … 993]",null
942,"[117, 200, … 259]",null
922,"[143, 89, … 596]",null
574,"[300, 258, … 754]",null
842,"[268, 269, … 333]",null
532,[269],"[58, 70, … 425]"
332,[350],"[566, 693, … 550]"


In [97]:
# строим индекс объектов
index = faiss.IndexFlatIP(item_embs.shape[1])
index.add(item_embs)
    
for user_id, test_ids, train_ids in grouped_user_items.rows():
    if train_ids:
        # используем инференс модели с помощью эмбеддинга пользователя
        user_history = train_ids
        
        y_rel = test_ids
        y_rec = [item_id for item_id in recs[user_id] if item_id not in user_history]
        hitrate_list.append(user_hitrate(y_rel, y_rec))
    else:
        # используем итеративный эмбеддинг по объектам
        user_emb = np.zeros(item_embs.shape[1])
        user_history = set()
        for i, item_ind in enumerate(test_ids[:-1]):
            if item_ind < len(item_embs):
                user_emb += item_embs[item_ind]
                
            user_history.add(item_ind)    
            y_rel = [test_ids[i + 1]]
            y_rec = [
                item_id 
                for item_id in index.search(user_emb[np.newaxis, :], TOP_K * 3)[1][0]
                if item_id not in user_history
            ]
    
        hitrate_list.append(user_hitrate(y_rel, y_rec, TOP_K))
print(f'Hitrate@{TOP_K} = {np.mean(hitrate_list)}')

Hitrate@10 = 0.1154877294287482


In [98]:
evaluate_holdout_set(train_df, user_embs, item_embs)

{'all': 0.060228452751817235,
 'warm': 0.20689655172413793,
 'cold': 0.055674518201284794}

## Валидация по событиям

В такой валидации мы рассматриваем для пользователя все взаимодействия, отсортированные по времени и оставляем последние N взаимодействия в качестве валидации, а все что раньше – для тренировки. У этого способа есть такая же проблема с тем, что при обучении могут сказываться дата-лики, однако в этом случае мы учитываем как холодных, так и теплых пользователей

In [100]:
grouped_user_items = (
    ratings
    .sort('timestamp')
    .groupby('user_id')
    .agg([
        pl.col('item_id').apply(lambda x: x[:-1]).alias('train_item_ids'),
        pl.col('item_id').apply(lambda x: [x[-1]]).alias('test_item_ids')
    ])
)

# sanity check
assert len(
    grouped_user_items
    .filter(pl.col('test_item_ids').apply(lambda x: len(x) == 0))
) == 0

grouped_user_items

user_id,train_item_ids,test_item_ids
i64,list[i64],list[i64]
174,"[315, 268, … 709]",[238]
298,"[286, 172, … 252]",[820]
760,"[258, 288, … 1135]",[375]
900,"[294, 237, … 483]",[618]
624,"[258, 286, … 272]",[313]
426,"[332, 492, … 505]",[185]
822,"[333, 902, … 71]",[111]
108,"[294, 319, … 281]",[290]
480,"[272, 302, … 642]",[504]


In [101]:
train_df = (
    grouped_user_items
    .select('user_id', 'train_item_ids')
    .explode('train_item_ids')
    .rename({'train_item_ids': 'item_id'})
)

user_embs, item_embs = fit_als(train_df)
probs, recs = get_recommendations(user_embs, item_embs)

100%|██████████| 15/15 [00:05<00:00,  2.94it/s]


In [102]:
hitrate_list = []
for user_id, user_history, y_rel in grouped_user_items.rows():
    y_rec = [
        item_id
        for item_id in recs[user_id]
        if item_id not in user_history
    ]
    hitrate_list.append(user_hitrate(y_rel, y_rec))
print(f'Hitrate@{TOP_K} = {np.mean(hitrate_list)}')

Hitrate@10 = 0.09330406147091108


In [103]:
evaluate_holdout_set(train_df, user_embs, item_embs)

{'all': 0.07352941176470588,
 'warm': 0.2318840579710145,
 'cold': 0.06709829311359623}

## Approximate KNN с помощью Qdrant

В отдельном процессе нужно запустить серверную часть [qdrant](https://qdrant.tech/) следующей командой:

`docker run -p 6333:6333 qdrant/qdrant`

In [74]:
client = QdrantClient("localhost", port=6333)
client.recreate_collection(
    collection_name="item_embs",
    # задаем размерность векторов и метрику дистанции
    vectors_config=VectorParams(size=item_embs.shape[1], distance=Distance.DOT),
)

True

In [75]:
operation_info = client.upsert(
    collection_name="item_embs",
    wait=True,
    points=[
        PointStruct(id=(item_id + 1), vector=item_emb.tolist())
        for item_id, item_emb in enumerate(item_embs[1:])
    ]
)
operation_info

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [76]:
user_id = 183
search_result = client.search(
    collection_name="item_embs",
    query_vector=user_embs[user_id].tolist(),
    limit=TOP_K
)
search_result

[ScoredPoint(id=222, version=0, score=0.33572915, payload={}, vector=None),
 ScoredPoint(id=228, version=0, score=0.31374466, payload={}, vector=None),
 ScoredPoint(id=202, version=0, score=0.30496037, payload={}, vector=None),
 ScoredPoint(id=227, version=0, score=0.28887057, payload={}, vector=None),
 ScoredPoint(id=405, version=0, score=0.2654244, payload={}, vector=None),
 ScoredPoint(id=230, version=0, score=0.23762143, payload={}, vector=None),
 ScoredPoint(id=229, version=0, score=0.18529022, payload={}, vector=None),
 ScoredPoint(id=196, version=0, score=0.18061997, payload={}, vector=None),
 ScoredPoint(id=380, version=0, score=0.17917344, payload={}, vector=None),
 ScoredPoint(id=151, version=0, score=0.16398694, payload={}, vector=None)]

In [77]:
y_rel = holdout_set.filter(pl.col('user_id') == user_id)['item_id'].to_list()
y_rec = [s.id for s in search_result]
user_hitrate(y_rel, y_rec)

1

## watched filter с разреженной матрицей

Для работы с redis нужно запустить docker контейнер следующей командой:

`docker run --rm -p 6379:6379 redis/redis-stack-server:latest`

In [78]:
# сгенерируем данные
n_interactions = 10_000
n_users = 10_000
n_items = 1_000_000

# wanna generate sparse matrix
assert n_interactions <= n_users * n_items / 100

interactions = set()

for _ in range(n_interactions):
    while True:
        user = np.random.choice(n_users)
        item = np.random.choice(n_items)

        if (item, user) not in interactions:
            interactions.add((item, user))
            break

In [79]:
r = redis.Redis(host='localhost', db=0)
used_memory_before = r.info('memory')['used_memory']

In [ ]:
for item, user in interactions:
    r.set(f'{item}-{user}', 1)

In [ ]:
r.info('memory')['used_memory'] - used_memory_before

Использовали ~500Кб памяти

In [37]:
for item, user in interactions:
    assert r.get(f'{item}-{user}') is not None
    
get_time_elapsed = []
for _ in range(n_interactions):
    user = np.random.choice(n_users)
    item = np.random.choice(n_items)
    
    t_start = time.time()
    get_result = r.get(f'{item}-{user}')
    get_time_elapsed.append((time.time() - t_start) * 1_000)
    
    if (item, user) in interactions:
        assert get_result is not None
        
print(f'{np.mean(get_time_elapsed):.4f} ± {np.std(get_time_elapsed):.4f} ms')

0.4972 ± 0.3755 ms


## Watched filter с помощью redis

https://redis-py.readthedocs.io/en/stable/redismodules.html

In [38]:
r = redis.Redis(db=1)
used_memory_before = r.info('memory')['used_memory']

In [39]:
# создадим БД watched_filter, которая будет расчитана на предопределенное число интеракций
# и делать в среднем 0.1% (вероятность делится на 100) ложноположительных срабатываний
r.bf().reserve('watched_filter', 0.001, n_interactions)

for item, user in interactions:
    r.bf().add('watched_filter', f'{item}-{user}')
    
for item, user in interactions:
    assert r.bf().exists('watched_filter', f'{item}-{user}')

In [40]:
r.info('memory')['used_memory'] - used_memory_before

20032

Для хранения используем в ~ 30 раз меньше памяти

Теперь проверим, изменилось ли время работы и сколько ложноположительных срабатываний произошло

In [41]:
num_errors = 0
get_time_elapsed = []

for _ in range(n_interactions):
    user = np.random.choice(n_users)
    item = np.random.choice(n_items)
    
    t_start = time.time()
    get_result = r.bf().exists('watched_filter', f'{item}-{user}')
    get_time_elapsed.append((time.time() - t_start) * 1_000)
    
    if (item, user) in interactions and get_result == 0:
        raise Exception('У bloom filter не бывает ложноотрицательных срабатываний')
    if (item, user) not in interactions and get_result == 1:
        num_errors += 1
        
print(f'Количество ложноположительных ошибок: {num_errors}')
print(f'{np.mean(get_time_elapsed):.4f} ± {np.std(get_time_elapsed):.4f} ms')

Количество ложноположительных ошибок: 5
0.5422 ± 0.4093 ms
